In [1]:
import random
from gx_utils import *
import numpy as np

In [2]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [3]:
import logging


def greedy(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    while goal != covered:
        x = all_lists.pop(0)
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)

    logging.info(
        f"Greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")

In [4]:

def depth_first(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    frontier = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    def build_solution_recursive(solution,covered,all_lists):
        #return solution
        #print(f"solution:{solution}")
        if goal == covered: #terminal condition
            #print("found!")
            return solution
        for i,x in enumerate(all_lists):
            next_all_lists = all_lists.copy() #create a copy of the list[list]
            next_list = next_all_lists.pop(i) #then pop the next list
            if not set(next_list) < covered: #if next list add some new number to my solution
                #make a (shallow) copy of my state and update that one
                next_covered = covered.copy()
                next_covered |= set(next_list)
                next_solution = solution.copy()
                next_solution.append(next_list)
                return build_solution_recursive(next_solution,next_covered,next_all_lists)
        return list() #no solution

    solution = build_solution_recursive(solution,covered,all_lists)
    logging.info(
        f"depth first solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")




In [5]:

def breadth_first(N):
    goal = set(range(N))
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    def build_solution(goal, all_lists):
        node_visited = 0
        covered = set()
        solution = list()
        frontier = list() # ( solution , covered, all_lists)
        #first step needed for initial state in recursive function
        for i,x in enumerate(all_lists):
                next_all_lists = all_lists.copy() #create a copy of the list[list]
                next_list = next_all_lists.pop(i) #then pop the next list
                if not set(next_list) < covered: #if next list add some new number to my solution
                    #make a (shallow) copy of my state and update that one
                    next_covered = covered.copy()
                    next_covered |= set(next_list)
                    next_solution = solution.copy()
                    next_solution.append(next_list)
                    frontier.append((next_solution,next_covered,next_all_lists))
        while len(frontier):
            (solution,covered,all_lists) = frontier.pop(0)
            node_visited +=1
            if covered == goal:
                return (solution,node_visited)
            for i,x in enumerate(all_lists):
                next_all_lists = all_lists.copy() #create a copy of the list[list]
                next_list = next_all_lists.pop(i) #then pop the next list
                if not set(next_list) < covered: #if next list add some new number to my solution
                    #make a (shallow) copy of my state and update that one
                    next_covered = covered.copy()
                    next_covered |= set(next_list)
                    next_solution = solution.copy()
                    next_solution.append(next_list)
                    frontier.append((next_solution,next_covered,next_all_lists))
        return (list(),node_visited)
        
    

    (solution,node_visited) = build_solution(goal, all_lists)

    logging.info(
        f"breadth first solution for N={N}: visiting {node_visited} nodes, w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")

In [18]:

from types import new_class


def a_star(N):

    def total_cost(state):
        g = state[0]
        h = N-len(state[1])
        return g + h

    node_visited = 0
    goal = set(range(N))
    covered = frozenset()
    solution = list()
    frontier = PriorityQueue()
    state = covered
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    #print(all_lists)
    for x_list in all_lists: #initial state
        new_state = (len(x_list),frozenset(x_list))
        cost = lambda s: total_cost(s)
        if new_state not in frontier:
            frontier.push(new_state,p = cost(new_state))
            #print(f"add to frontier:{new_state} with cost:{cost(new_state)}")

    state = frontier.pop()
    cost = lambda s: total_cost(s)
    while state[1] != goal and frontier is not None:
        for x_list in all_lists:
            if frozenset(x_list) not in state[1]: #the list add some coverage to the current state
                #new_covered = frozenset().union(state[1],)
                #print(new_covered)
                new_state = (state[0]+len(x_list),state[1] | frozenset(x_list))
                if new_state not in frontier:
                    frontier.push(new_state,p = cost(new_state))
                    #print(f"add to frontier:{new_state} with cost:{cost(new_state)}")
                #else:
                #    print(f"{x_list} already in frontier")
            #else:
            #    print(f"{x_list} doesn't add new number to {state[1]}")
        state = frontier.pop()  
        node_visited += 1  
        #print(f"expanding: {state} with cost:{cost(state)}")
    logging.info(
    f"A* solution for N={N} visiting {node_visited} nodes: w={w} (bloat={(w-N)/N*100:.0f}%)"
    )
    logging.debug(f"{state[1]}")
    return (state)
N=5
#(w,solution)= a_star(5)
#logging.info(
#    f"A* solution for N={N} visiting {node_visited} nodes: w={w} (bloat={(w-N)/N*100:.0f}%)"
#)
#logging.debug(f"{solution}")
 

INFO:root:A* solution for N=5 visiting 30 nodes: w=5 (bloat=0%)


In [19]:
logging.getLogger().setLevel(logging.INFO)
for N in [5, 10, 20]:
    greedy(N)
    a_star(N)

INFO:root:Greedy solution for N=5: w=5 (bloat=0%)
INFO:root:Greedy solution for N=10: w=13 (bloat=30%)
INFO:root:Greedy solution for N=20: w=46 (bloat=130%)


In [7]:
%timeit greedy(1_000)

INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)


874 ms ± 161 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
